In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_timestamp, concat
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType
from config_file import processed_path, transformed_path

In [2]:
spark = SparkSession.builder.appName('ETL').getOrCreate()
spark

## Read all the processed files

### 1. Read Circuit processed file

In [3]:
circuits_df = spark.read.parquet(f"{processed_path}\Circuits").withColumnRenamed('location','circuit_Location')
circuits_df.show()

+----------+--------------+--------------------+----------------+---------+--------+---------+--------+--------------------+
|circuit_Id|   circuit_Ref|                name|circuit_Location|  country|latitude|longitude|altitude|      ingestion_Date|
+----------+--------------+--------------------+----------------+---------+--------+---------+--------+--------------------+
|         1|   albert_park|Albert Park Grand...|       Melbourne|Australia|-37.8497|  144.968|      10|2022-06-13 10:08:...|
|         2|        sepang|Sepang Internatio...|    Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|2022-06-13 10:08:...|
|         3|       bahrain|Bahrain Internati...|          Sakhir|  Bahrain| 26.0325|  50.5106|       7|2022-06-13 10:08:...|
|         4|     catalunya|Circuit de Barcel...|        Montmeló|    Spain|   41.57|  2.26111|     109|2022-06-13 10:08:...|
|         5|      istanbul|       Istanbul Park|        Istanbul|   Turkey| 40.9517|   29.405|     130|2022-06-13 10:08:...|


### 2. Read Race processed file

In [4]:
race_df = spark.read.parquet(f"{processed_path}\Race").withColumnRenamed('name','race_Name').withColumnRenamed('race_Timestamp','race_Date')
race_df.show()

+-------+-----+----------+--------------------+-------------------+--------------------+---------+
|race_Id|round|circuit_Id|           race_Name|          race_Date|      ingestion_Date|race_Year|
+-------+-----+----------+--------------------+-------------------+--------------------+---------+
|   1053|    2|        21|Emilia Romagna Gr...|2021-04-18 13:00:00|2022-06-13 10:54:...|     2021|
|   1052|    1|         3|  Bahrain Grand Prix|2021-03-28 15:00:00|2022-06-13 10:54:...|     2021|
|   1051|   21|         1|Australian Grand ...|2021-11-21 06:00:00|2022-06-13 10:54:...|     2021|
|   1054|    3|        20|                 TBC|               null|2022-06-13 10:54:...|     2021|
|   1055|    4|         4|  Spanish Grand Prix|2021-05-09 13:00:00|2022-06-13 10:54:...|     2021|
|   1056|    5|         6|   Monaco Grand Prix|2021-05-23 13:00:00|2022-06-13 10:54:...|     2021|
|   1057|    6|        73|Azerbaijan Grand ...|2021-06-06 12:00:00|2022-06-13 10:54:...|     2021|
|   1058| 

### 3. Read Drivers processed file

In [5]:
driver_df = spark.read.parquet(f"{processed_path}\Drivers").withColumn('driver_Name',concat(col('first_Name'),lit(' '),col('last_Name')))\
.withColumnRenamed('number','driver_Number').withColumnRenamed('nationality','driver_Nationality')

driver_df = driver_df.drop('first_Name','last_Name')
driver_df.show()

+---------+----------+----------+----+-------------+------------------+--------------------+------------------+
|driver_Id|driver_Ref|       dob|code|driver_Number|driver_Nationality|      ingestion_Date|       driver_Name|
+---------+----------+----------+----+-------------+------------------+--------------------+------------------+
|        1|  hamilton|1985-01-07| HAM|           44|           British|2022-06-13 10:35:...|    Lewis Hamilton|
|        2|  heidfeld|1977-05-10| HEI|            0|            German|2022-06-13 10:35:...|     Nick Heidfeld|
|        3|   rosberg|1985-06-27| ROS|            6|            German|2022-06-13 10:35:...|      Nico Rosberg|
|        4|    alonso|1981-07-29| ALO|           14|           Spanish|2022-06-13 10:35:...|   Fernando Alonso|
|        5|kovalainen|1981-10-19| KOV|            0|           Finnish|2022-06-13 10:35:...| Heikki Kovalainen|
|        6|  nakajima|1985-01-11| NAK|            0|          Japanese|2022-06-13 10:35:...|   Kazuki Na

### 4. Read Constructors processed file

In [6]:
constructor_df = spark.read.parquet(f"{processed_path}\Constructors").withColumnRenamed('name','team').withColumnRenamed('nationality','constructor_Nationality')
constructor_df.show()

+--------------+---------------+-----------+-----------------------+--------------------+
|constructor_Id|constructor_Ref|       team|constructor_Nationality|      ingestion_Date|
+--------------+---------------+-----------+-----------------------+--------------------+
|             1|        mclaren|    McLaren|                British|2022-06-13 10:14:...|
|             2|     bmw_sauber| BMW Sauber|                 German|2022-06-13 10:14:...|
|             3|       williams|   Williams|                British|2022-06-13 10:14:...|
|             4|        renault|    Renault|                 French|2022-06-13 10:14:...|
|             5|     toro_rosso| Toro Rosso|                Italian|2022-06-13 10:14:...|
|             6|        ferrari|    Ferrari|                Italian|2022-06-13 10:14:...|
|             7|         toyota|     Toyota|               Japanese|2022-06-13 10:14:...|
|             8|    super_aguri|Super Aguri|               Japanese|2022-06-13 10:14:...|
|         

### 5. Read Results processed file

In [7]:
result_df = spark.read.parquet(f"{processed_path}\Results").withColumnRenamed('time','race_Time')
result_df.show()

+--------------+---------+-----------+-----------------+----------------+----+----+------------+------+------+--------+--------------+-------------+----+---------+-----------+--------------------+-------+
|constructor_Id|driver_Id|fastest_Lap|fastest_Lap_Speed|fastest_Lap_Time|grid|laps|milliseconds|number|points|position|position_Order|position_Text|rank|result_Id|  race_Time|      ingestion_Date|race_Id|
+--------------+---------+-----------+-----------------+----------------+----+----+------------+------+------+--------+--------------+-------------+----+---------+-----------+--------------------+-------+
|             1|       18|         52|              0.0|        1:36.568|   2|  53|     5453427|     4|  25.0|       1|             1|            1|   1|    21112|1:30:53.427|2022-06-13 10:52:...|    855|
|             6|        4|         50|          216.481|        1:36.682|   5|  53|     5454587|     5|  18.0|       2|             2|            2|   4|    21113|     +1.160|2022-

## Transformations on the read data

In [8]:
### 1. Join Race and Circuits df

race_circuit_df = race_df.join(circuits_df, race_df.circuit_Id==circuits_df.circuit_Id,'inner')

In [9]:
### 2. Join Results df with all other df

final_df = result_df.join(race_circuit_df, result_df.race_Id==race_circuit_df.race_Id,'inner')\
                        .join(driver_df, result_df.driver_Id==driver_df.driver_Id,'inner')\
                        .join(constructor_df, result_df.constructor_Id==constructor_df.constructor_Id,'inner')

In [11]:
final_df = final_df.select('race_Year','race_Name','race_Date','circuit_Location','driver_Name','driver_Number','driver_Nationality','team','grid','race_Time','fastest_Lap','points','position')

In [12]:
final_df = final_df.withColumn('created_Date',current_timestamp())

In [13]:
final_df.show()

+---------+--------------------+-------------------+----------------+-----------------+-------------+------------------+------------+----+-----------+-----------+------+--------+--------------------+
|race_Year|           race_Name|          race_Date|circuit_Location|      driver_Name|driver_Number|driver_Nationality|        team|grid|  race_Time|fastest_Lap|points|position|        created_Date|
+---------+--------------------+-------------------+----------------+-----------------+-------------+------------------+------------+----+-----------+-----------+------+--------+--------------------+
|     2018|Australian Grand ...|2018-03-25 05:10:00|       Melbourne|  Sergey Sirotkin|           35|           Russian|    Williams|  19|         \N|          3|   0.0|    null|2022-06-13 18:28:...|
|     2018|Australian Grand ...|2018-03-25 05:10:00|       Melbourne|  Marcus Ericsson|            9|           Swedish|      Sauber|  17|         \N|          4|   0.0|    null|2022-06-13 18:28:...|


In [14]:
final_df.write.mode('overwrite').parquet(f"{transformed_path}\Race_Results")

## Data Validation

In [15]:
final_df.filter((final_df['race_Year']==2020) & (final_df['race_Name']=='Abu Dhabi Grand Prix')).orderBy(final_df['points'].desc()).show()

+---------+--------------------+-------------------+----------------+------------------+-------------+------------------+------------+----+-----------+-----------+------+--------+--------------------+
|race_Year|           race_Name|          race_Date|circuit_Location|       driver_Name|driver_Number|driver_Nationality|        team|grid|  race_Time|fastest_Lap|points|position|        created_Date|
+---------+--------------------+-------------------+----------------+------------------+-------------+------------------+------------+----+-----------+-----------+------+--------+--------------------+
|     2020|Abu Dhabi Grand Prix|2020-12-13 13:10:00|       Abu Dhabi|    Max Verstappen|           33|             Dutch|    Red Bull|   1|1:36:28.645|         14|  25.0|       1|2022-06-13 18:28:...|
|     2020|Abu Dhabi Grand Prix|2020-12-13 13:10:00|       Abu Dhabi|   Valtteri Bottas|           77|           Finnish|    Mercedes|   2|    +15.976|         40|  18.0|       2|2022-06-13 18:28: